## Setup

In [1]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.AI.DocumentIntelligence, 1.0.0"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Microsoft.ML.Tokenizers, 1.0.2"
#r "nuget: Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2"
#r "nuget: pdf2image-dotnet, 1.0.0"
#r "nuget: SkiaSharp, 3.116.1"
#r "nuget: SkiaSharp.NativeAssets.Linux, 3.116.1"

#!import ../../modules/samples/AppSettings.csx
#!import ../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx
#!import ../../modules/samples/helpers/StopwatchContext.csx
#!import ../../modules/samples/models/Classification.csx
#!import ../../modules/samples/models/invoice.csx
#!import ../../modules/samples/models/DocumentProcessingResult.csx
#!import ../../modules/samples/evaluation/AccuracyEvaluator.csx
#!import ../../modules/samples/confidence/ConfidenceHelpers.csx
#!import ../../modules/samples/confidence/OpenAIConfidence.csx
#!import ../../modules/samples/confidence/DocumentIntelligenceConfidence.csx

using System;
using System.IO;
using System.Text.Json;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;
using Azure.AI.DocumentIntelligence;
using OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using Pdf2Image;
using SkiaSharp;

Installed Packages Azure.AI.DocumentIntelligence, 1.0.0 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.13.2 DotNetEnv, 3.1.1 Microsoft.ML.Tokenizers, 1.0.2 Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2 pdf2image-dotnet, 1.0.0 SkiaSharp, 3.116.1 SkiaSharp.NativeAssets.Linux, 3.116.1

Loading extensions from `/home/vscode/.nuget/packages/skiasharp/3.116.1/interactive-extensions/dotnet/SkiaSharp.DotNet.Interactive.dll`

In [ ]:
string workingDir = Path.GetFullPath("../../../../");
AppSettings settings = new AppSettings(new Dictionary<string, string>(Env.Load(Path.Combine(workingDir, ".env"))));
string samplePath = Path.Combine(workingDir, "samples/dotnet/scenarios/invoices/");
string sampleName = "invoice-extraction";

DefaultAzureCredential credential = new DefaultAzureCredential(
    new DefaultAzureCredentialOptions { 
        ExcludeWorkloadIdentityCredential = true,
        ExcludeAzureDeveloperCliCredential = true,
        ExcludeEnvironmentCredential = true,
        ExcludeManagedIdentityCredential = true,
        ExcludeAzurePowerShellCredential = true,
        ExcludeSharedTokenCacheCredential = true,
        ExcludeInteractiveBrowserCredential = true
    }
);

AzureOpenAIClient openaiClient = new AzureOpenAIClient(
    new Uri(settings.AzureOpenAIEndpoint),
    credential
);

var documentIntelligenceClient = new DocumentIntelligenceClient(
    new Uri(settings.AzureAIServicesEndpoint),
    credential
);

In [3]:
string inputsPath = Path.Combine(workingDir, "samples/assets/invoices/");
string metadataFName = "invoice_6.json";
string metadataFPath = Path.Combine(inputsPath, metadataFName);

Dictionary<string, JsonElement> data = JsonSerializer.Deserialize<Dictionary<string, JsonElement>>(File.ReadAllText(metadataFPath));

List<Invoice> expectedOutputs = new List<Invoice>();

foreach (var key in data.Keys)
{
    if (key.EndsWith("_expected"))
    {
        string invoiceKey = key.Substring(0, key.Length - 9);
        string json = data[key].ToString();
        Invoice invoice = JsonSerializer.Deserialize<Invoice>(
            json, 
            new JsonSerializerOptions(JsonSerializerDefaults.Web)
            {
                PropertyNamingPolicy = JsonNamingPolicy.SnakeCaseLower
            });
        expectedOutputs.Add(invoice);
    }
}

string pdfFName = data["fname"].GetString();
string pdfFPath = Path.Combine(inputsPath, pdfFName);

AccuracyEvaluator<Invoice> invoiceEvaluator = new AccuracyEvaluator<Invoice>(matchKeys: new List<string>() { "ProductCode", "Description" });

In [4]:
List<ChatMessageContentPart> pageImages = new List<ChatMessageContentPart>();
List<byte[]> pages = new List<byte[]>();

StopwatchContext imageSw;

using (imageSw = new StopwatchContext())
{
    var pdfBytes = File.ReadAllBytes(pdfFPath);
    pages = await Pdf2ImageConverter.FromBytesAsync(pdfBytes);

    for (int i = 0; i < pages.Count; i++)
    {
        var pageData = BinaryData.FromBytes(pages[i]);
        pageImages.Add(ChatMessageContentPart.CreateImagePart(pageData, "image/png"));
    }
}

In [5]:
List<ClassificationDefinitionModel> classifications = new List<ClassificationDefinitionModel>()
{
    new ClassificationDefinitionModel() {
        Classification = "Invoice/Credit Note",
        Description = "A document that serves as a bill for goods or services provided, often used for payment processing and record-keeping."
    }
};

In [6]:
StringBuilder classifySystemPromptBuilder = new StringBuilder();
classifySystemPromptBuilder.AppendLine("You are an AI assistant that helps detect the boundaries of sub-section or sub-documents using the provided classifications.");
classifySystemPromptBuilder.AppendLine("## Classifications");
classifySystemPromptBuilder.AppendLine(JsonSerializer.Serialize(classifications));

string classifySystemPrompt = classifySystemPromptBuilder.ToString();

In [7]:
List<ChatMessageContentPart> classifyUserContent = new List<ChatMessageContentPart>();

In [8]:
StringBuilder classifyUserPromptBuilder = new StringBuilder();
classifyUserPromptBuilder.AppendLine("Classify documents that are an invoice/credit note in the provided page images.");
classifyUserPromptBuilder.AppendLine("- A single classification may span multiple page images.");
classifyUserPromptBuilder.AppendLine("- A single page image may contain multiple classifications.");
classifyUserPromptBuilder.AppendLine("- If a page image does not contain a classification, ignore it.");

string classifyUserPrompt = classifyUserPromptBuilder.ToString();

classifyUserContent.Add(ChatMessageContentPart.CreateTextPart(classifyUserPrompt));

for (int i = 0; i < pageImages.Count; i++)
{
    classifyUserContent.Add(ChatMessageContentPart.CreateTextPart($"Page {i + 1}"));
    classifyUserContent.Add(pageImages[i]);
}

In [ ]:
ParsedChatCompletion<ClassificationsModel> classifyCompletion;

StopwatchContext classifySw;

using (classifySw = new StopwatchContext())
{
    classifyCompletion = await openaiClient
        .GetChatClient(settings.AzureOpenAIChatDeployment)
        .CompleteChatAsync(
            [
                new SystemChatMessage(classifySystemPrompt),
                new UserChatMessage(classifyUserContent)
            ],
            new ChatCompletionOptions
            {   
                ResponseFormat = CreateJsonSchemaFormat<ClassificationsModel>("classificationsModel", jsonSchemaIsStrict: true),
                MaxOutputTokenCount = 4096,
                Temperature = 0.1f,
                TopP = 0.1f,
                IncludeLogProbabilities = true
            }
        );
}

In [10]:
var documentClassifications = classifyCompletion.Parsed;

In [11]:
// DEBUG: Display the document classifications
Console.WriteLine(JsonSerializer.Serialize(documentClassifications, new JsonSerializerOptions(JsonSerializerDefaults.Web) { WriteIndented = true }));

{
  "classifications": [
    {
      "classification": "Invoice/Credit Note",
      "imageRangeStart": 1,
      "imageRangeEnd": 2
    }
  ]
}


In [17]:
var classifiedImages = new Dictionary<int, IEnumerable<ChatMessageContentPart>>();
var classifiedImagesBytes = new Dictionary<int, IEnumerable<byte[]>>();

for (int i = 0; i < documentClassifications.Classifications.Count; i++)
{
    var classification = documentClassifications.Classifications[i];
    var imageRangeStart = classification.ImageRangeStart ?? 1;
    var imageRangeEnd = classification.ImageRangeEnd ?? 1;

    classifiedImages.Add(i, pageImages.GetRange(imageRangeStart - 1, imageRangeEnd));
    classifiedImagesBytes.Add(i, pages.GetRange(imageRangeStart - 1, imageRangeEnd));
}

## Extract

In [ ]:
for (int i = 0; i < classifiedImages.Count; i++)
{
    Console.WriteLine($"Classification {i}");

    var images = classifiedImages[i];
    var imageBytes = classifiedImagesBytes[i];

    var boundPdfFPath = Path.Combine(samplePath, $"{sampleName}_{pdfFName}_{i}.pdf");

    using (var pdfStream = new SKFileWStream(boundPdfFPath))
    using (var pdfDocument = SKDocument.CreatePdf(pdfStream))
    {
        foreach (var pageImage in imageBytes)
        {
            using var input = new SKMemoryStream(pageImage);
            using var bitmap = SKBitmap.Decode(input);

            using var canvas = pdfDocument.BeginPage(bitmap.Width, bitmap.Height);
            canvas.DrawBitmap(bitmap, 0, 0);
            pdfDocument.EndPage();
        }
        pdfDocument.Close();
    }

    AnalyzeResult result;

    StopwatchContext diSw;

    using (diSw = new StopwatchContext())
    {
        var pollerResult = await documentIntelligenceClient.AnalyzeDocumentAsync(
            WaitUntil.Completed,
            options: new AnalyzeDocumentOptions(
                modelId: "prebuilt-layout",
                bytesSource: BinaryData.FromBytes(File.ReadAllBytes(boundPdfFPath)))
            {
                OutputContentFormat = DocumentContentFormat.Markdown
            }
        );

        result = pollerResult.Value;
    }

    string markdown = result.Content;

    // DEBUG: Display the document intelligence markdown content
    Console.WriteLine(markdown);

    string extractSystemPrompt = "You are an AI assistant that extracts data from documents.";

    // Prepare the user content for the OpenAI API including any specific details for processing this type of document, text, and the document page images.
    List<ChatMessageContentPart> extractUserContent = new List<ChatMessageContentPart>();

    StringBuilder extractUserPromptBuilder = new StringBuilder();
    extractUserPromptBuilder.AppendLine("Extract the data from this invoice. ");
    extractUserPromptBuilder.AppendLine("- If a value is not present, provide null.");
    extractUserPromptBuilder.AppendLine("- It is possible that there are multiple invoices in the same document across multiple pages.");
    extractUserPromptBuilder.AppendLine("- Some values must be inferred based on the content defined in the invoice.");
    extractUserPromptBuilder.AppendLine("- Dates should be in the format YYYY-MM-DD.");

    string extractUserPrompt = extractUserPromptBuilder.ToString();

    extractUserContent.Add(ChatMessageContentPart.CreateTextPart(extractUserPrompt));
    extractUserContent.Add(ChatMessageContentPart.CreateTextPart(markdown));

    extractUserContent.AddRange(images);

    ParsedChatCompletion<Invoice> completion;

    StopwatchContext oaiSw;

    using (oaiSw = new StopwatchContext())
    {
        completion = await openaiClient
            .GetChatClient(settings.AzureOpenAIChatDeployment)
            .CompleteChatAsync(
                [
                    new SystemChatMessage(extractSystemPrompt),
                    new UserChatMessage(extractUserContent)
                ],
                new ChatCompletionOptions
                {   
                    ResponseFormat = CreateJsonSchemaFormat<Invoice>("invoice", jsonSchemaIsStrict: true),
                    MaxOutputTokenCount = 4096,
                    Temperature = 0.1f,
                    TopP = 0.1f,
                    IncludeLogProbabilities = true
                }
            );
    }

    // Gets the parsed Invoice object from the completion response.
    var invoice = completion.Parsed;

    var accuracy = invoiceEvaluator.Evaluate(expectedOutputs[i], invoice);

    // Determines the confidence of the extracted data.
    var diConfidence = DocumentIntelligenceConfidence<Invoice>.EvaluateConfidence(invoice, result);
    var oaiConfidence = OpenAIConfidence<Invoice>.EvaluateConfidence(invoice, completion.Origin);

    var confidence = ConfidenceHelpers.MergeConfidenceValues(diConfidence, oaiConfidence);

    // DEBUG: Gets the total execution time of the data extraction process.
    var totalElapsed = classifySw.Elapsed + diSw.Elapsed + imageSw.Elapsed + oaiSw.Elapsed;

    // DEBUG: Gets the prompt tokens and completion tokens from the completion response.
    var promptTokens = completion.Usage.InputTokenCount;
    var completionTokens = completion.Usage.OutputTokenCount;

    // DEBUG: Save the output of the data extraction result.
    var extractionResult = new DataProcessingResult<Invoice>(
        invoice,
        accuracy,
        confidence,
        promptTokens,
        completionTokens,
        totalElapsed
    );

    var extractionResultJson = JsonSerializer.Serialize(extractionResult, new JsonSerializerOptions { WriteIndented = true });
    var extractionResultFPath = Path.Combine(samplePath, $"{sampleName}.{pdfFName}_{i}.json");

    await File.WriteAllTextAsync(extractionResultFPath, extractionResultJson);

    // DEBUG: Display the outputs of the data extraction process.
    var output = new
    {
        Document = pdfFName,
        Index = i,
        Accuracy = $"{float.Parse(accuracy["overall"].ToString()) * 100:0.00}%",
        Confidence = $"{float.Parse(confidence["_overall"].ToString()) * 100:0.00}%",
        ExecutionTime = $"{totalElapsed.TotalSeconds:0.00} seconds",
        ClassifyExecutionTime = $"{classifySw.Elapsed.TotalSeconds:0.00} seconds",
        DocumentIntelligenceExecutionTime = $"{diSw.Elapsed.TotalSeconds:0.00} seconds",
        ImagePreprocessingExecutionTime = $"{imageSw.Elapsed.TotalSeconds:0.00} seconds",
        OpenAIExecutionTime = $"{oaiSw.Elapsed.TotalSeconds:0.00} seconds",
        PromptTokens = promptTokens,
        CompletionTokens = completionTokens,
    };

    display(output);
    display(confidence);
}

Classification 0
NEXGEN
Innovation Drives Progress

Unit 4, Innovation Drive

Milton Keynes

Buckinghamshire

MK9 1FH

Phone: +44 (0)1234 567890
info@nexgen.co.uk

BILL TO

AlphaTech Solutions

22 Regent Street

London

SWIY 4PD

SHIP TO

AlphaTech Solutions

Warehouse 5, Met Business Park

Leicester

LE19 1WD


<table>
<tr>
<th>Invoice No.</th>
<th>Invoice Date</th>
<th>Ship Date</th>
<th>Purchase Order</th>
<th>Customer Ref.</th>
</tr>
<tr>
<td>NG-INV-2025-00123</td>
<td>2025-04-24</td>
<td>2025-04-19</td>
<td>PO-556423</td>
<td>ALTEC-SLN</td>
</tr>
</table>


<table>
<tr>
<th>Product Code</th>
<th>Description</th>
<th>Units</th>
<th>Excl.</th>
<th>VAT</th>
<th>Incl.</th>
</tr>
<tr>
<td>10323</td>
<td>MAILBOX 25PK</td>
<td>245</td>
<td>3787.70</td>
<td>568.16</td>
<td>☒ 4355.86</td>
</tr>
</table>


15%

COMMENTS
Expect payment by 6 May!

REMIT PAYMENT TO

NEXGEN

Unit 4, Innovation Drive

Milton Keynes

Buckinghamshire

MK9 1FH


<table>
<tr>
<td>Name:</td>
<td>Julia Patel</td>
</tr

Document,invoice_6.pdf
Index,0
Accuracy,96.49%
Confidence,97.09%
ExecutionTime,48.96 seconds
ClassifyExecutionTime,11.83 seconds
DocumentIntelligenceExecutionTime,9.58 seconds
ImagePreprocessingExecutionTime,15.29 seconds
OpenAIExecutionTime,12.25 seconds
PromptTokens,5357
CompletionTokens,348


key type value CustomerName System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999902701418591 value System.String AlphaTech Solutions CustomerTaxId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String CustomerAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Double 0.9998972616738169 value System.String 22 Regent Street City key type value confidence System.Double 0.99993674964885 value System.String London State key type value confidence System.Double 0 value System.String PostalCode key type value confidence System.Double 0.9906075080810421 value System.String SWIY 4PD Country key type value confidence System.Double 0.6879837306074783 value System.String UK ShippingAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Double 0.9995697832264213 value System.String Warehouse 5, Met Business Park City key type value confidence System.Double 0.9999020641783191 value System.String Leicester State key type value confidence System.Double 0 value System.String PostalCode key type value confidence System.Double 0.999850701820876 value System.String LE19 1WD Country key type value confidence System.Double 0.9996709061098533 value System.String UK PurchaseOrder System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9998638757895516 value System.String PO-556423 InvoiceId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999510135541327 value System.String NG-INV-2025-00123 InvoiceDate System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999619413029649 value System.String 2025-04-24 DueDate System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9808994411075389 value System.String 2025-05-06 VendorName System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9930528451973027 value System.String NEXGEN VendorAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Double 0.9983620224292696 value System.String Unit 4, Innovation Drive City key type value confidence System.Double 0.9998002677878554 value System.String Milton Keynes State key type value confidence System.Double 0.9998159489270185 value System.String Buckinghamshire PostalCode key type value confidence System.Double 0.9998314712733932 value System.String MK9 1FH Country key type value confidence System.Double 0.9977359769899904 value System.String UK VendorTaxId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String RemittanceAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Double 0.9998330681434422 value System.String Unit 4, Innovation Drive City key type value confidence System.Double 0.9999613838185623 value System.String Milton Keynes State key type value confidence System.Double 0.9999619400808779 value System.String Buckinghamshire PostalCode key type value confidence System.Double 0.9999716508835553 value System.String MK9 1FH Country key type value confidence System.Double 0.9999043288264341 value System.String UK Subtotal System.Collections.Generic.Dictionary<System.String,System.Object> key value CurrencyCode key type value confidence System.Double 0.9921935753229044 value System.String GBP Amount key type value confidence System.Double 0.9387103821563516 value System.String 3787.7 TotalDiscount System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 